<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT19/blob/tobzed/Lab-3/tedwards_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Iterative Methods**
**Tobias Edwards**

# **Abstract**

This lab focused on iterative methods for solving linear equation systems of the form $Ax = b$. These methods included:
Jacobi iteration, Gauss-Seidel iteration and the Generalized minimal residual method. This lab also looked at solving nonlinear equations and nonlinear systems of equation such that a continuous function $f(x) = 0$. 

#**About the code**

The following code is authored by Tobias Edwards for the third lab in course DD2363  Methods in Scientific Computing.



# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import numpy as np
import unittest

# **Introduction**

The Jacobi method and the Gauss-Sediel method require that the matrix $A$ is diagonally dominant for the methods to converge. Informally, this means that each element on the diagonal has a magnitude at least as large as the sum of the magnitudes of the other elements on the corresponding row. These methods are called "fixed-point iterative" as they find a solution $x^{(k+1)}$ by applying the same algorithm on the previous approximation $x^{(k)}$. The generalized minimal residual (gmres) method works by creating an orthonormal Krylov basis and then solving a least squares problem.Newton's method for scalar equations functions by finding the dervivative of a point, creating a line from that derivative, and finding where this line intersects the x-axis. This point of intersection will be our new approximation. The method for solvning nonlinear systems is a generalization of Newton's method for scalar equations where the derivative is replaced by the jacobian matrix.  



# **Results**

## Jacobi method

This method solves $Ax = b$ by the iteration $x^{(k+1)} = (I-A)x^{(k)} + b$, a form of Richardson's iteration such that $M = I - A$. In this algorithm we split $A = A_1 + A_2$such that $A_1$is a diagonal matrix and $A_2$ is equal to $A-D$. $A_1$ is now easy to invert. 

In [0]:
def jacobi_iteration(A,b,TOL):
    n  = A.shape[0]
    x = np.zeros(n)
    r_norm = np.linalg.norm(b-A.dot(x))
    while r_norm >= TOL:
        x_old = np.copy(x)
        for i in range(n):
            val = 0.0
            for j in range(n):
                if j != i:
                    val = val + A[i,j]*x_old[j]
            x[i] = (b[i]-val)/A[i,i]
        r_norm = np.linalg.norm(b-A.dot(x))
    return x

## Gauss-Seidel method

This method instead splits $A$ into two triangular matrices which can be inverted my forward/backward substitution. 

In [0]:
def gauss_seidel_iteration(A,b,TOL):
    x = np.zeros(A.shape[0])
    n,m = A.shape
    r_norm = np.linalg.norm(b-A.dot(x))
    while r_norm >= TOL:
        x_next = np.zeros_like(x)
        for i in range(n):
            val = b[i]
            for j in range(n):
                if j < i:
                    val -= A[i,j]*x_next[j]
                if j > i:
                    val -= A[i,j]*x[j]
            x_next[i] = np.float(val/A[i,i])
        x = x_next.copy()
        r_norm = np.linalg.norm(b-A.dot(x_next))
    return x

## Newton's method for scalar nonlinear equation

In [0]:
def newtons_method(f,df,TOL,x=None):
    if x == None:
        x = np.random.rand()
    iteration = 0
    while np.linalg.norm(f(x)) >= TOL and iteration < 500:
        x -= f(x)/df(x)
    if iteration == 500:
        return None
    return x

## GMRES 

The arnoldi method is used to create an orthonormal basis consisting of $k$ vectors $<b, Ab, A^2b...A^{k-1}b>$, and also creates an upper Hessenberg matrix. The Hessenberg matrix is used to solve a least squares problem and then the $k+1$ iteration of $x$ is given by $Qy$ where $y$ is the solution to the least squares problem. 

In [0]:
def arnoldi_iteration(A,b,k):
    Q = np.zeros((A.shape[0],k+1))
    H = np.zeros((k+1,k))
    Q[:,0] = b/np.linalg.norm(b)
    for i in range(k):
        v = A.dot(Q[:,i])
        for j in range(i+1):
            H[j,i] = np.inner(Q[:,j],v)
            v -= H[j,i]*Q[:,j]
        H[i+1,i] = np.linalg.norm(v)
        Q[:,i+1] = v / H[i+1,i]
    return Q,H

def gmres(A,b,TOL):
    x = np.zeros(A.shape[0])
    b_norm = np.linalg.norm(b)
    k = 1
    while np.linalg.norm(b-A.dot(x)) >= TOL:
        Q,H = arnoldi_iteration(A,b,k) # this gives us Q_{k+1} and correspinding upper Hessenberg
        e1 = np.zeros(k+1)
        e1[0] = 1
        y = np.linalg.lstsq(H,b_norm*e1,rcond=-1)[0]
        x = Q[:,:k].dot(y)
        k = k + 1
    return x

##Newton's method for vector nonlinear equation

In [0]:
def newtons_systems(f, jacobian, dim, TOL):
    x = np.random.rand(dim)
    while np.linalg.norm(f(x)) >= TOL:
        grad = gmres(jacobian(x),-f(x),TOL)
        x += grad
    return x

## Tests

In [39]:
class Lab3FunctionsTest(unittest.TestCase):

    def test_jacobi_iteration(self):
        TOL_list = [1, 1e-2, 1e-4, 1e-7]
        A = np.array([ # diagonally dominat matrix
                    [10.,1.,-2.,3.],
                    [-1.,15.,4.,1.],
                    [0.,0.,4.,-1.],
                    [10.,-7.,2.,80.]
                    ])
        b = np.array([0.,4.,-2.,1.])
        for TOL in TOL_list:
            x = jacobi_iteration(A,b,TOL)
            self.assertTrue(np.linalg.norm(A.dot(x)-b) < TOL)
        A2 = np.array([[3,0,-1], [0,-2,1], [0,0,6]])
        b2 = np.array([4,0,1])
        y = np.array([25./18., 1./12., 1./6.]) # exact solution
        x2 = jacobi_iteration(A2,b2,TOL_list[3])
        self.assertTrue(np.linalg.norm(x2-y) < TOL_list[3])

    def test_gauss_seidel_iteration(self):
        TOL_list = [1, 1e-2, 1e-4, 1e-7]
        A = np.array([ # diagonally dominant matrix
                    [10.,1.,-2.,3.],
                    [-1.,15.,4.,1.],
                    [0.,0.,4.,-1.],
                    [10.,-7.,2.,80.]
                    ])
        b = np.array([0.,4.,-2.,1.])
        for TOL in TOL_list:
            x = gauss_seidel_iteration(A,b,TOL)
            self.assertTrue(np.linalg.norm(A.dot(x)-b) < TOL)
        A2 = np.array([[3,0,-1], [0,-2,1], [0,0,6]])
        b2 = np.array([4,0,1])
        y = np.array([25./18., 1./12., 1./6.]) # exact solution
        x2 = gauss_seidel_iteration(A2,b2,TOL_list[3])
        self.assertTrue(np.linalg.norm(x2-y) < TOL_list[3])

    def test_newtons_method(self):
        f = lambda x: x**3 - 2*x - 4
        df = lambda x: 3*x**2 - 2
        TOL_list = [1, 1e-2, 1e-4, 1e-7]
        for TOL in TOL_list:
            x = newtons_method(f,df,TOL)
            self.assertTrue(f(x) < TOL)
            self.assertTrue(np.linalg.norm(f(x)-f(2.)) < TOL)
            
    def test_gmres(self):
        A = np.array([[1,0,-2,4],[-1,2,5,3], [-7,8,2,9], [0,8,2,-1]])
        b = np.array([1,-2,5,3])
        TOL_list = [1, 1e-2, 1e-4, 1e-7]
        for TOL in TOL_list:
            x = gmres(A,b,TOL)
            self.assertTrue(np.linalg.norm(b-A.dot(x)) < TOL)
        A2 = np.array([[-2,3],[8,2]])
        b2 = np.array([1,-1])
        y = np.array([-10./56., 3./14.])
        for TOL in TOL_list:
            x2 = gmres(A2,b2,TOL)
            self.assertTrue(np.linalg.norm(y-x2) < TOL)

    def test_newtons_systems(self):
        f = lambda x: np.array([
                                (x[0]**2)*x[1] + 3*x[2] - x[1]**3 + 1,
                                3*x[1]-x[2]**2+x[0],
                                x[0]**3-7*x[1]+x[2]
                              ])
        df = lambda x: np.array([
                            [ 2*x[0]*x[1], x[0]**2-3*x[1]**2, 3 ],
                            [ 1, 3, -2*x[2] ],
                            [3*x[0]**2, -7, 1 ]
                            ])
        TOL_list = [1, 1e-2, 1e-4, 1e-7]
        for TOL in TOL_list:
            x = newtons_systems(f,df,3,TOL)
            self.assertTrue(np.linalg.norm(f(x)) < TOL)
        
        f2 = lambda x: np.array([x[0]-3,x[1]+2,x[2]])
        df2 = lambda x: np.array([[1,0,0], [0,1,0], [0,0,1]])
        y = np.array([3,-2,0]) # exact solution
        for TOL in TOL_list:
            x2 = newtons_systems(f2,df2,3,TOL)
            self.assertTrue(np.linalg.norm(x2-y) < TOL)
            
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..../usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
.
----------------------------------------------------------------------
Ran 5 tests in 0.027s

OK


# Discussion

The theory for this lab was easier to understand than last weeks theory. However, I found this weeks programming to be quite challenging. Also, I had to make sure that the arguments would result in convergence for some algorithms; something that I forgot and spent a long time trying to figure out why my algorithms were not working. 